In [ ]:
import numpy as npimport numpy as np
import matplotlib.pyplot as plt

# ----------------------------
# Problem parameters (yours)
# ----------------------------
L = 20.0
c = 1.0

sigma = 0.4
amplitude = 3.0
x0 = L/4
t_final = 10.0

nx = 200
# Periodic-friendly grid: endpoint=False so 0 and L are not both included
x  = np.linspace(0.0, L, nx, endpoint=False)
dx = L / nx

r  = 0.2                        # Courant number
dt = r * dx / abs(c)
nt = int(np.round(t_final/dt))  # number of steps
dt = t_final/nt                 # hit t_final exactly
t  = np.linspace(0.0, t_final, nt+1)

# ----------------------------
# WENO5-JS reconstruction & RK3
# ----------------------------
def weno5_left(u, eps=1e-6):
    um2 = np.roll(u,  2); um1 = np.roll(u, 1); u0 = u
    up1 = np.roll(u, -1); up2 = np.roll(u, -2)

    # 3rd-order candidates at i+1/2
    p0 = ( 1/3)*um2 + (-7/6)*um1 + (11/6)*u0
    p1 = (-1/6)*um1 + ( 5/6)*u0  + ( 1/3)*up1
    p2 = ( 1/3)*u0  + ( 5/6)*up1 + (-1/6)*up2

    # Jiang–Shu smoothness indicators
    beta0 = (13/12)*(um2 - 2*um1 + u0)**2 + 0.25*(um2 - 4*um1 + 3*u0)**2
    beta1 = (13/12)*(um1 - 2*u0  + up1)**2 + 0.25*(um1 - up1)**2
    beta2 = (13/12)*(u0  - 2*up1 + up2)**2 + 0.25*(3*u0  - 4*up1 + up2)**2

    g0, g1, g2 = 0.1, 0.6, 0.3
    a0 = g0 / (eps + beta0)**2
    a1 = g1 / (eps + beta1)**2
    a2 = g2 / (eps + beta2)**2
    den = a0 + a1 + a2 + 1e-300  # tiny guard against divide-by-zero
    w0, w1, w2 = a0/den, a1/den, a2/den
    return w0*p0 + w1*p1 + w2*p2

def rhs_weno5(u, c, dx):
    # c>0 upwind; for general f(u) use LF splitting instead
    uL = weno5_left(u)      # left state at i+1/2, stored at index i
    F  = c * uL
    return -(F - np.roll(F, 1)) / dx  # conservative flux difference

def step_ssprk3(u, c, dx, dt):
    k1 = rhs_weno5(u, c, dx);  u1 = u + dt*k1
    k2 = rhs_weno5(u1, c, dx); u2 = 0.75*u + 0.25*(u1 + dt*k2)
    k3 = rhs_weno5(u2, c, dx); u3 = (1/3)*u + (2/3)*(u2 + dt*k3)
    return u3

# ----------------------------
# IC (yours), history storage
# ----------------------------
# IC & history for WENO
u_WENO = np.zeros((nt+1, nx))
u_WENO[0, :] = amplitude * np.exp(-((x - x0)**2) / (2.0 * sigma**2))

uw = u_WENO[0, :].copy()          # <-- was "u" before; now "uw"
for n in range(nt):
    uw = step_ssprk3(uw, c, dx, dt)
    u_WENO[n+1, :] = uw

# ----------------------------
# Plot snapshots (same style)
# ----------------------------
plt.figure(figsize=(9, 3))
plt.plot(x, u_WENO[0, :],        label='t=0')
plt.plot(x, u_WENO[nt//4, :],    label=f't≈{t[nt//4]:.2f}')
plt.plot(x, u_WENO[nt//2, :],    label=f't≈{t[nt//2]:.2f}')
plt.plot(x, u_WENO[-1, :],       label='t=final')
plt.xlabel('x'); plt.ylabel('u'); plt.legend(); plt.grid(True)
plt.title('WENO5 + SSPRK3 snapshots')
plt.tight_layout(); plt.show()

# Optional sanity checks
mass0 = u_WENO[0].sum()*dx
massf = u_WENO[-1].sum()*dx
print(f"Mass: initial={mass0:.8f}, final={massf:.8f}, diff={massf-mass0:+.2e}")
print("All finite:", np.isfinite(u_WENO).all())

import matplotlib.pyplot as plt

# ----------------------------
# Problem parameters (yours)
# ----------------------------
L = 20.0
c = 1.0

sigma = 0.4
amplitude = 3.0
x0 = L/4
t_final = 10.0

nx = 200
# Periodic-friendly grid: endpoint=False so 0 and L are not both included
x  = np.linspace(0.0, L, nx, endpoint=False)
dx = L / nx

r  = 0.2                        # Courant number
dt = r * dx / abs(c)
nt = int(np.round(t_final/dt))  # number of steps
dt = t_final/nt                 # hit t_final exactly
t  = np.linspace(0.0, t_final, nt+1)

# ----------------------------
# WENO5-JS reconstruction & RK3
# ----------------------------
def weno5_left(u, eps=1e-6):
    um2 = np.roll(u,  2); um1 = np.roll(u, 1); u0 = u
    up1 = np.roll(u, -1); up2 = np.roll(u, -2)

    # 3rd-order candidates at i+1/2
    p0 = ( 1/3)*um2 + (-7/6)*um1 + (11/6)*u0
    p1 = (-1/6)*um1 + ( 5/6)*u0  + ( 1/3)*up1
    p2 = ( 1/3)*u0  + ( 5/6)*up1 + (-1/6)*up2

    # Jiang–Shu smoothness indicators
    beta0 = (13/12)*(um2 - 2*um1 + u0)**2 + 0.25*(um2 - 4*um1 + 3*u0)**2
    beta1 = (13/12)*(um1 - 2*u0  + up1)**2 + 0.25*(um1 - up1)**2
    beta2 = (13/12)*(u0  - 2*up1 + up2)**2 + 0.25*(3*u0  - 4*up1 + up2)**2

    g0, g1, g2 = 0.1, 0.6, 0.3
    a0 = g0 / (eps + beta0)**2
    a1 = g1 / (eps + beta1)**2
    a2 = g2 / (eps + beta2)**2
    den = a0 + a1 + a2 + 1e-300  # tiny guard against divide-by-zero
    w0, w1, w2 = a0/den, a1/den, a2/den
    return w0*p0 + w1*p1 + w2*p2

def rhs_weno5(u, c, dx):
    # c>0 upwind; for general f(u) use LF splitting instead
    uL = weno5_left(u)      # left state at i+1/2, stored at index i
    F  = c * uL
    return -(F - np.roll(F, 1)) / dx  # conservative flux difference

def step_ssprk3(u, c, dx, dt):
    k1 = rhs_weno5(u, c, dx);  u1 = u + dt*k1
    k2 = rhs_weno5(u1, c, dx); u2 = 0.75*u + 0.25*(u1 + dt*k2)
    k3 = rhs_weno5(u2, c, dx); u3 = (1/3)*u + (2/3)*(u2 + dt*k3)
    return u3

# ----------------------------
# IC (yours), history storage
# ----------------------------
# IC & history for WENO
u_WENO = np.zeros((nt+1, nx))
u_WENO[0, :] = amplitude * np.exp(-((x - x0)**2) / (2.0 * sigma**2))

uw = u_WENO[0, :].copy()          # <-- was "u" before; now "uw"
for n in range(nt):
    uw = step_ssprk3(uw, c, dx, dt)
    u_WENO[n+1, :] = uw

# ----------------------------
# Plot snapshots (same style)
# ----------------------------
plt.figure(figsize=(9, 3))
plt.plot(x, u_WENO[0, :],        label='t=0')
plt.plot(x, u_WENO[nt//4, :],    label=f't≈{t[nt//4]:.2f}')
plt.plot(x, u_WENO[nt//2, :],    label=f't≈{t[nt//2]:.2f}')
plt.plot(x, u_WENO[-1, :],       label='t=final')
plt.xlabel('x'); plt.ylabel('u'); plt.legend(); plt.grid(True)
plt.title('WENO5 + SSPRK3 snapshots')
plt.tight_layout(); plt.show()

# Optional sanity checks
mass0 = u_WENO[0].sum()*dx
massf = u_WENO[-1].sum()*dx
print(f"Mass: initial={mass0:.8f}, final={massf:.8f}, diff={massf-mass0:+.2e}")
print("All finite:", np.isfinite(u_WENO).all())
